In [1]:
from dotenv import load_dotenv
import os
import warnings
import logging

# Set logging level to INFO
logging.basicConfig(level=logging.INFO)

warnings.filterwarnings("ignore")

# Load variables from .env file
load_dotenv()

True

In [2]:
import getpass
import os

if not os.environ.get("MISTRALAI_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

model.invoke("Hello, how are you!")


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content="I'm just an AI, I don't have feelings like humans do, but I'm functioning properly and ready to help you with any questions or tasks you have! It's great to chat with you. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 16, 'total_tokens': 65, 'completion_time': 0.040833333, 'prompt_time': 0.002372446, 'queue_time': 0.0007843940000000003, 'total_time': 0.043205779}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-473ee1a7-132d-443d-a7f5-36ae87ff46f4-0', usage_metadata={'input_tokens': 16, 'output_tokens': 49, 'total_tokens': 65})

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader, DirectoryLoader

def load_pdf(file_path):
    return PyMuPDFLoader(file_path).load()

documents = load_pdf("input/gen_ai_langchain_2024.pdf")


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def split_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
        add_start_index=True,
    )
    
    # Split the documents into chunks
    return text_splitter.split_documents(documents)

chunks = split_documents(documents)

# add unique ids to each chunk combining source, page and start_index
for chunk in chunks:
    source = chunk.metadata.get("source")
    page = chunk.metadata.get("page")
    start_index = chunk.metadata.get("start_index")
    chunk.metadata["id"] = f"{source}_{page}_{start_index}"

print(f"Number of Documents: {len(documents)} and Number of Chunks: {len(chunks)}")

Number of Documents: 361 and Number of Chunks: 1014


In [5]:
type(chunks[3])

langchain_core.documents.base.Document

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

def get_embedding_function():
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# test the embedding function
embedding_function = get_embedding_function()
len(embedding_function.embed_query("Hello, how are you!"))

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


768

In [13]:
from langchain.vectorstores.chroma import Chroma
from langchain_core.documents import Document
CHROMA_PATH = "chroma_db"
def add_to_chroma(chunks: list[Document]):
    db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=get_embedding_function(),
    )
    
    # add or update the chunks in the chroma db
    existing_items = db.get(include=[])
    if existing_items is None or "id" not in existing_items:
        existing_items = {"id": []}
    existing_ids = set(existing_items["id"])
    
    print(f"Number of existing items: {len(existing_ids)}")
    
    new_chunks = [chunk for chunk in chunks if chunk.metadata["id"] not in existing_ids]
    new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
    db.add_documents(new_chunks, ids=new_chunk_ids)
    db.persist()
    
try:
    add_to_chroma(chunks)
except Exception as e:
    print(e)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


C:\temp\ipykernel_9492\3526497029.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Number of existing items: 0


: 

In [27]:
PROMPT_TEMPLATE = """
Answer the question based on only on the following context:
```
{context}
```

Answer the question based on the context above: 
```
{question}
```
"""

In [29]:
# import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

def query_rag(query_text: str, model):
    db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=get_embedding_function(),
    )
    
    results = db.similarity_search_with_score(query_text, k=5)
    
    context_text = "\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    
    response = model.invoke(prompt)
    
    sources = [doc.metadata.get("id") for doc, _score in results]